<a href="https://colab.research.google.com/github/HWMV/OrchestrAI/blob/main/backend_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## File : agent_component.py

In [ ]:
from pydantic import BaseModel
from typing import List

# tool은 [] 리스트 형태로 받음
class Tool(BaseModel):
    name: str
    description: str

class AgentComponent(BaseModel):
    role: str
    goal: str
    backstory: str
    tools: List[Tool]

## File : agents.py

In [ ]:
from crewai import Agent
from .agent_component import AgentComponent
from .tools import Tools

class Agents:
    @staticmethod
    def create_agent(component: AgentComponent) -> Agent:
        return Agent(
            role=component.role,
            goal=component.goal,
            backstory=component.backstory,
            verbose=True,
            tools=[Tools.create_tool(tool.name) for tool in component.tools]
        )

## File : tasks.py

In [ ]:
from crewai import Task, Agent
from typing import Dict, Any

class Tasks:
    @staticmethod
    def create_task(task_data: Dict[str, Any], agent: Agent) -> Task:
        return Task(
            description=task_data['description'],
            expected_output=task_data['expected_output'],
            agent=agent,
            output_file=task_data['file_name.format'],
        )

## File : tools.py

In [ ]:
class Tools:
    @staticmethod
    def create_tool(tool_name: str):
        # 원하는 Tool 로직 구현
        pass

## File : crews.py

In [ ]:
from crewai import Crew, Agent, Task
from typing import List
from langchain_openai import ChatOpenAI
# 추후 Gemini 추가 : 이전에 구현한 방법 있음, 참고 요망

class CrewExecutor:
    def __init__(self, agents: List[Agent], tasks: List[Task]):
        self.agents = agents
        self.tasks = tasks

    def execute(self, inputs: Dict[str, Any] = None):
        crew = Crew(
            agents=self.agents,
            tasks=self.tasks,
            verbose=2,
            process=Process.hierarchical,
            manager_llm=ChatOpenAI(model="gpt-4"),
            memory=True,
        )
        return crew.kickoff(inputs=inputs)

## File : main.py

In [ ]:
# 사용자의 입력을 받았다고 가정하고 테스트 예제 추가함
from .agents import Agents
from .tasks import Tasks
from .agent_component import AgentComponent, Tool
from .crews import CrewExecutor

class Tasks:
    @staticmethod
    def create_task(task_data: Dict[str, Any], agent: Agent) -> Task:
        return Task(
            description=task_data['description'],
            expected_output=task_data['expected_output'],
            agent=agent,
            output_file=task_data['file_name.format'],
        )

# file: tool_factory.py
class ToolFactory:
    @staticmethod
    def create_tool(tool_name: str):
        # Here you would implement the logic to create and return the actual tool
        # This could involve importing the necessary functions or classes
        pass

# file: crew_executor.py
from crewai import Crew, Agent, Task
from typing import List
from langchain_openai import ChatOpenAI

class CrewExecutor:
    def __init__(self, agents: List[Agent], tasks: List[Task]):
        self.agents = agents
        self.tasks = tasks

    def execute(self, inputs: Dict[str, Any] = None):
        crew = Crew(
            agents=self.agents,
            tasks=self.tasks,
            verbose=2,
            process=Process.hierarchical,
            manager_llm=ChatOpenAI(model="gpt-4"),
            memory=True,
        )
        return crew.kickoff(inputs=inputs)

# file: main.py
from .agents import Agents
from .tasks import Tasks
from .agent_component import AgentComponent, Tool
from .crews import CrewExecutor

# Define your agents
financial_analyst = AgentComponent(
    role="Financial Analyst",
    goal="Evaluate a stock's financial health and performance",
    backstory="You're a very experienced investment advisor who uses a combination of technical and fundamental analysis to provide strategic investment advice to your clients.",
    tools=[
        Tool(name="Balance Sheet", description="Retrieve balance sheet data", expected_output="str", output_file="file_name.format"),
        Tool(name="Income Statement", description="Retrieve income statement data", expected_output="str", output_file="file_name.format"),
        Tool(name="Insider Transactions", description="Retrieve insider trading data", expected_output="str", output_file="file_name.format"),
    ]
)

# Create agents
agent = AgentFactory.create_agent(financial_analyst)

# Define tasks
tasks = [
    {"description": "Analyze the company's financial statements"},
    {"description": "Evaluate recent insider trading activities"},
    {"description": "Provide a comprehensive financial health report"}
]

# Create tasks
created_tasks = [TaskFactory.create_task(task, agent) for task in tasks]

# Create and execute crew
executor = CrewExecutor([agent], created_tasks)
# input Example
result = executor.execute(inputs={"company": "Salesforce"})
print(result)